<a href="https://colab.research.google.com/github/shubham311994/colab_notebooks/blob/main/pytorch_rnn_based_qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


data = pd.read_csv('100_Unique_QA_Dataset.csv')
data.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# tokenize the data
def tokenizer(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'", '')
  return text.split()

In [ ]:
# form the vocabulary
vocab = {'<UNK>':0}

In [ ]:
def build_vocab(row):

  tokenized_question = tokenizer(row['question'])
  tokenized_answer = tokenizer(row['answer'])
  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [ ]:
data.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
len(vocab)

324

In [ ]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []
  for token in tokenizer(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [ ]:
text_to_indices('What is campusx', vocab)

[1, 2, 0]

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab


  def __len__(self):
    return self.df.shape[0]


  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [ ]:
dataset = QADataset(data, vocab)

In [ ]:
dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

In [ ]:
# for question, answer in dataloader:
#   print(question, answer)

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.full_connected_layer = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.full_connected_layer(final.squeeze(0))

    return output

In [ ]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [ ]:
learning_rate = 0.001
epochs = 20

In [ ]:
model = SimpleRNN(len(vocab))

In [ ]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss
    loss = criteria(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    # updating loss

    total_loss = total_loss + loss.item()

  print( f'Epoch: {epoch + 1}, Loss: {total_loss:4f}')

Epoch: 1, Loss: 523.491996
Epoch: 2, Loss: 451.459094
Epoch: 3, Loss: 372.195901
Epoch: 4, Loss: 311.832979
Epoch: 5, Loss: 261.329476
Epoch: 6, Loss: 213.164889
Epoch: 7, Loss: 169.763209
Epoch: 8, Loss: 132.208582
Epoch: 9, Loss: 101.293239
Epoch: 10, Loss: 77.563632
Epoch: 11, Loss: 60.208783
Epoch: 12, Loss: 47.274283
Epoch: 13, Loss: 38.092056
Epoch: 14, Loss: 31.066339
Epoch: 15, Loss: 25.519567
Epoch: 16, Loss: 21.370881
Epoch: 17, Loss: 18.274452
Epoch: 18, Loss: 15.473800
Epoch: 19, Loss: 13.530291
Epoch: 20, Loss: 11.725567


In [ ]:
def predict(model, question, threshold=0.5):
  # convert question
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # model
  output = model(question_tensor)

  # convert logits to probabilites
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print(f"I don't know")

  print(list(vocab.keys())[index])

In [ ]:
predict(model, "What is the largest planet in our solar system")

jupiter
